In [ ]:
%matplotlib inline
## Run this cell once before the lab to download
## the mnist dataset and the pre-trained ResNet50 model. 

## Mnist
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

## Keras pre-trained weights
from keras.utils.data_utils import get_file
get_file('resnet50_weights_tf_dim_ordering_tf_kernels.h5',
         'https://github.com/fchollet/deep-learning-models/releases'+
         '/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels.h5',
         cache_subdir='models',
         md5_hash='a7b3fe01876f51b976af0dea6bc144eb')

# Convolutional Neural Networks

Objectives:
- TensorFlow tutorial
- Application of convolution on images
- First conv net on MNIST with TensorFlow
- Use a pre-trained ResNet with Keras for transfer learning (second notebook)

Home assignment: fine-tuning a resnet on GPU (third notebook)

## TensorFlow 101

TensorFlow is a symbolic graph computation engine, that allows automatic differentiation of each node
- https://www.tensorflow.org 
- https://www.tensorflow.org/tutorials/mnist/tf/

TensorFlow builds where nodes may be:
- **constant:** constants tensors, such as a learning rate
- **Variables:** any tensor, such as parameters of the models
- **Placeholders:** placeholders for inputs and outputs of your models
- many other types of nodes (functions, loss, ...)

The graph is symbolic, no computation is performed until a `Session` is defined and the command `run` or `eval` is invoked. TensorFlow may run this computation on (multiple) CPUs or GPUs

In [ ]:
import tensorflow as tf

a = tf.constant(3)
b = tf.constant(2)
c = a + b

In [ ]:
print(type(a))
print(a)

In [ ]:
print(type(c))
print(c)

In [ ]:
with tf.Session() as sess:
    c_value = sess.run(c)
    
print(type(c_value))
print(c_value)

In [ ]:
d = tf.Variable(0)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(d))

    sess.run(d.assign_add(c))
    print(sess.run(d))

    sess.run(d.assign_add(c))
    print(sess.run(d))

#### Input placeholders

- The placeholder is a variable that doesn't have a value yet in the symbolic graph. The value will be fed when running the session by passing the `feed_dict` argument
- If the placeholder is a k-dimensional tensor, we need to specify its shape. 
- It is possible to leave the shape variable by putting `None` values in the shape

In [ ]:
x = tf.placeholder("float32", name="input")
y = x + tf.constant(3.0)

with tf.Session() as sess:
    print(sess.run(y, feed_dict={x: 2}))

In [ ]:
import numpy as np

img = tf.placeholder("float32", shape=(1, 2, 3), name="input")
inverted_image = 255. - img

with tf.Session() as sess:
    fake_img = np.zeros(shape=(1, 2, 3))
    print(sess.run(inverted_image, feed_dict={img:fake_img}))

In [ ]:
img = tf.placeholder("float32", shape= [None, None, 3], name="input")
inverted_image = 255. - img
with tf.Session() as sess:
    fake_img = np.zeros(shape=(3, 2, 3))
    print(sess.run(inverted_image, feed_dict={img:fake_img}))

### Reading and opening images

The following code enables to read an image, put it in a numpy array and display it in the notebook.

In [ ]:
import tensorflow as tf
import numpy as np
from scipy.misc import imread, imresize
import matplotlib.pyplot as plt

In [ ]:
sample_image = imread("bumblebee.png")
sample_image= sample_image.astype(float)

size = sample_image.shape
print("sample image shape: "+ str(sample_image.shape))

def show(image):
    image = np.squeeze(image.astype("uint8"))
    plt.imshow(image, cmap="gray")

show(sample_image)

### A simple convolution filter

The goal of this section to use TensorFlow to perform convolutions on images. This section does not involve training any model yet.

We build a convolution filter that blurs the image using `tf.nn.depthwise_conv2d` (treats each channel separately)

In [ ]:
image = tf.placeholder(tf.float32, shape=(None, None, None, 3))
kernel = tf.placeholder(tf.float32, shape=(5, 5, 3))

def conv(x, k):
    k = tf.reshape(k, shape=(5, 5, 3, 1))
    return tf.nn.depthwise_conv2d(x, k, strides=(1, 1, 1, 1),
                                  padding='SAME')
    
output_image = conv(image, kernel)
kernel_data = np.zeros(shape=(5, 5, 3)).astype(np.float32)
kernel_data[:, :, :] = 1 / 25

# move the channel dimension to the first dimension to
# make it easy to see the spacial organization of the kernel
# on the last 2 dimensions with print:
print(np.transpose(kernel_data, (2, 0, 1))) 

In [ ]:
with tf.Session() as sess:
    feed_dict = {image: [sample_image], kernel: kernel_data}
    conv_img = sess.run(output_image, feed_dict=feed_dict)
    print(conv_img.shape)
    show(conv_img[0])

**Exercise**
- Build an identity 3x3 kernel with stride 2. What is the size of the output image?
- Change the padding to 'VALID'. What do you observe?

In [ ]:
# %load solutions/strides_padding.py

In [ ]:
# convert image to greyscale
grey_sample_image = sample_image.sum(axis=2) / 3.

# add the channel dimension even if it's only one channel
grey_sample_image = grey_sample_image[:, :, np.newaxis]

show(grey_sample_image)

**Exercise**
- Build an edge detector using `tf.nn.conv2d` on greyscale image
- You may experiment with several kernels to find a way to detect edges
- https://en.wikipedia.org/wiki/Kernel_(image_processing)

Try `tf.nn.conv2d?` or press `shift-tab` to get the documentation. You may get help at https://www.tensorflow.org/api_docs/python/nn/convolution

In [ ]:
# %load solutions/edge_detection

### Pooling and strides with convolutions

**Exercise**
- Use `tf.nn.max_pool` to apply a 2x2 max pool to the image
- Use `tf.nn.avg_pool` to apply an average pooling.
- Is it possible to compute a max pooling and an average pooling with well chosen kernels?

**bonus**
- Implement a 3x3 average pooling with a regular convolution `tf.nn.conv2d`, with well chosen strides, kernel and padding

In [ ]:
# %load solutions/pooling.py

In [ ]:
# %load solutions/average_as_conv.py

# Building a network on MNIST

https://www.tensorflow.org/tutorials/mnist/pros/

- Using Tensorflow
- Training data preprocessed
- Include regularization methods

### A simple feedforward model in TensorFlow 

- A logistic regression without taking into account the spatiality of the data
- Very similar to lab01

In [ ]:
# MNIST is 28x28 = 784 dimensions
x = tf.placeholder(tf.float32, shape=[None, 784])
y_true = tf.placeholder(tf.float32, shape=[None, 10])

W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

y_pred = tf.matmul(x,W) + b

# We don't have to do the softmax ourselves, TensorFlow can use
# logits directly to compute the loss
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true)
loss = tf.reduce_mean(cross_entropy)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
with tf.Session() as sess:
    # Initialize weights
    sess.run(tf.global_variables_initializer())

    # Train loop
    for i in range(1000):
        # mnist.train helper function builds a batch of N elements
        batch = mnist.train.next_batch(100)
        train_step.run(feed_dict={x: batch[0], y_true: batch[1]})
        
    feed_dict={x: mnist.test.images, 
               y_true: mnist.test.labels}
    print(accuracy.eval(feed_dict=feed_dict))

### CNN model in TensorFlow

You are going to build a convolutional neural network with TensorFlow.

The following helper functions were taken from TensorFlow tutorial https://www.tensorflow.org/tutorials/mnist/pros/

They allow:
- to define weights and bias by only specifying the shape
- easy use of convolutions and max_pool layers

In [ ]:
# Helper functions

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

In order to use the spatial geometry of the image, we reshape the input tensor to (`batch_size, 28, 28, channel_number`)

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_true = tf.placeholder(tf.float32, shape=[None, 10])

# only 1 channel (grey image) and the batch size is -1 because
# we don't know its size beforehand
x_image = tf.reshape(x, (-1, 28, 28, 1))
print(x_image.get_shape())

In [ ]:
with tf.Session() as sess:
    batch = mnist.train.next_batch(10)
    x_value, x_image_value = sess.run(
        [x, x_image], feed_dict={x: batch[0]})
    
    print(x_value.shape, x_image_value.shape)

In [ ]:
# Convolution layer example in TensorFlow
W_conv1 = weight_variable([5, 5, 1, 32])
h_conv1 = conv2d(x_image, W_conv1)
h_pool1 = max_pool_2x2(h_conv1)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batch = mnist.train.next_batch(10)
    output_conv, output_pool = sess.run(
        [h_conv1, h_pool1], feed_dict={x: batch[0]})
    
    print("conv activation shape:", output_conv.shape)
    print("pool activation shape:", output_pool.shape)

** Exercise **

Build a CNN with the following architecture:
- Convolution 5x5, 32 output channels + bias
- ReLU (you may use `tf.nn.relu`)
- Maxpool 2x2
- Convolution 5x5, 64 output channels + bias
- ReLU
- Maxpool 2x2
- Fully connected layer of size 1024 (you may use `tf.reshape(x, [-1, size])`)
- ReLU
- Output fully connected layer of size 10 
- The output should be named `y_conv`

**Bonus** add dropout
- A 50% dropout should work here
- You may use `tf.nn.dropout(x, keep_prob)`
- You should add keep prob as an input of the model, and pass 0.5 during training and 1.0 during test time

In [ ]:
# %load solutions/mnist_conv.py

In [ ]:
# softmax and loss
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_true)
loss = tf.reduce_mean(cross_entropy)

# optimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

# accuracy
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_true,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(500):
        batch = mnist.train.next_batch(50)
        if i%100 == 0:
            feed_dict = {x:batch[0], y_true: batch[1]}
            train_accuracy = accuracy.eval(feed_dict=feed_dict)
            print("step %d, training accuracy %g"
                  % (i, train_accuracy))
        feed_dict = {x: batch[0], y_true: batch[1]}
        train_step.run(feed_dict = feed_dict)
        
    feed_dict = {x: mnist.test.images[:1000],
                 y_true: mnist.test.labels[:1000]}
    print("test accuracy %g" % accuracy.eval(feed_dict=feed_dict))

In [ ]:
#%load solutions/mnist_conv_dropout.py